# Mask R-CNN - Apply model to hail imagery subsets

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
import glob
import os
import sys
import random
import math
import re
import time
import json
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage

from multiprocessing import Pool

# Root directory of the project
ROOT_DIR = os.path.abspath("/home/jsoderho/dev/Mask_RCNN-tfv2")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import hail

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = "/data/hailpixel_20200119"

#path to weights
HAIL_WEIGHTS_PATH = "/data/hailpixel_20200119/mask_rcnn_hail_final.h5"  # TODO: update this path

#path to subset images
SUBSET_PATH = '/data/hailpixel_20200119/testing_subset'
ROI_PATH    = '/data/hailpixel_20200119/testing_roi'

Using TensorFlow backend.


## Configurations

In [2]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

#load config
config = hail.HailConfig()

In [3]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        1000
DETECTION_MIN_CONFIDENCE       0.3
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE        

## Load Model

In [5]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [6]:
# Load hail training model weights
weights_path = HAIL_WEIGHTS_PATH #model.find_last()
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  /data/hailpixel_20200119/mask_rcnn_hail_final.h5


In [7]:
def process_image(subset_ffn):
    #read image
    image = skimage.io.imread(subset_ffn)
    #run model on image
    results  = model.detect([image], verbose=0)
    roi_list = results[0]['rois']
    #save back to file
    base = os.path.basename(subset_ffn)
    json_header = base + str(file_sz)
    #generate json header
    json_out = {json_header: {'filename': base,
                   'size': file_sz,
                   'regions':[],
                   'file_attributes':{}
                  }
                }
    #build json for every roi
    for roi in roi_list:
        #convert roi box to centroids
        cx = int((roi[1] + roi[3])/2)
        cy = int((roi[0] + roi[2])/2)
        #generate object for json
        json_out[json_header]['regions'].append({'shape_attributes':{'name':'point',
                                                        'cx': cx,
                                                        'cy': cy},
                                    'region_attributes':{}})
    #build json fn
    out_fn  = base[:-3] + 'json'
    out_ffn = '/'.join([ROI_PATH, out_fn])
    #write to file
    with open(out_ffn, 'w') as fp:
        json.dump(json_out, fp)
    #output to user
    print('finished', base, 'with number of detections', len(roi_list))

In [8]:
#list subsets
subset_ffn_list = sorted(glob.glob(SUBSET_PATH + '/*.png'))

for subset_ffn in subset_ffn_list:
    file_sz = os.path.getsize(subset_ffn)
    #check file size
    if file_sz/1000 < 50:
        continue
#     #check index number
#     base = os.path.basename(subset_ffn)
#     base_index = int(base[:-4])
#     if base_index<1128:
#         continue
    process_image(subset_ffn) #last 1706

print('Finished RCNN run')

finished 0325.png with number of detections 233
finished 0326.png with number of detections 232
finished 0327.png with number of detections 220
finished 0328.png with number of detections 232
Finished RCNN run
